In [1]:
import numpy as np
import math

# My name is XXXX --> 4
L = 4
d_k = 8
d_v = 8

In [2]:
q = np.random.randn(L, d_k)
k = np.random.randn(L, d_k)
v = np.random.randn(L, d_k)

In [3]:
q, k, v

(array([[ 0.61273083,  1.13588303, -0.07668819, -0.11967717, -1.10663414,
          0.72558852,  0.32466577,  1.01415271],
        [-0.43102019,  1.70190526, -0.27558612, -0.24166775,  0.04095081,
          1.04663879, -1.05256731,  0.17359994],
        [ 0.06644399, -1.20945125, -0.24851811, -0.88769826, -1.0417973 ,
         -0.06049551, -0.27082861,  0.31942523],
        [-0.34797781,  1.03270325, -0.21377292,  1.36688   ,  0.56332009,
         -1.53889172, -0.18758204,  1.19075472]]),
 array([[ 0.05174536, -1.17034938, -1.27582011,  1.40638064, -0.87467139,
         -0.61068974,  1.50818084, -0.08137509],
        [ 1.5182199 ,  0.29325469,  0.37417365, -0.39781567,  0.95559851,
          0.73178876,  0.05184995,  0.43025465],
        [ 0.25307245, -1.30778832, -0.94060822, -0.5155473 ,  1.293209  ,
          1.33223228,  1.47178584, -0.86005869],
        [-1.23924754, -1.46183783, -0.81514191, -0.59448902, -1.81793514,
          0.34715823,  0.15822686, -1.02893737]]),
 array([[-1.

# Self-Attention Working

In [4]:
np.matmul(q, k.T)

array([[-0.43618567,  1.20893535, -2.05544562, -1.01458911],
       [-4.27898422,  0.66289808, -2.20213727, -1.64172268],
       [ 1.00126395, -0.91006745,  0.18874811,  3.91738292],
       [ 1.03571951, -0.93445041, -4.56410572, -4.5299648 ]])

In [5]:
q.var(), k.var()

(0.6474576452070601, 0.9672822038086253)

In [6]:
np.matmul(q, k.T).var()

5.02958295435073

In [7]:
scaled = np.matmul(q, k.T) / math.sqrt(d_k)

In [8]:
scaled

array([[-0.15421492,  0.42742319, -0.72670977, -0.35871142],
       [-1.51284938,  0.23436986, -0.7785731 , -0.58043662],
       [ 0.35400027, -0.32175743,  0.06673254,  1.38500401],
       [ 0.36618215, -0.33037811, -1.61365505, -1.60158441]])

In [9]:
q.var(), k.var(), scaled.var()

(0.6474576452070601, 0.9672822038086253, 0.6286978692938413)

# Masking

In [10]:
mask = np.tril(np.ones((L, L)))
mask

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [11]:
mask[ mask == 0] = -np.inf
mask[ mask == 1] = 0

In [12]:
mask

array([[  0., -inf, -inf, -inf],
       [  0.,   0., -inf, -inf],
       [  0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.]])

In [13]:
scaled + mask

array([[-0.15421492,        -inf,        -inf,        -inf],
       [-1.51284938,  0.23436986,        -inf,        -inf],
       [ 0.35400027, -0.32175743,  0.06673254,        -inf],
       [ 0.36618215, -0.33037811, -1.61365505, -1.60158441]])

# Softmax

In [14]:
def softmax(x):
    return (np.exp(x).T / np.sum(np.exp(x), axis=-1)).T

In [15]:
attention = softmax(scaled + mask)
attention

array([[1.        , 0.        , 0.        , 0.        ],
       [0.14839828, 0.85160172, 0.        , 0.        ],
       [0.44265776, 0.22521133, 0.33213091, 0.        ],
       [0.56301337, 0.28054752, 0.07774748, 0.07869163]])

In [16]:
new_v = np.matmul(attention, v)
new_v

array([[-1.05032823e+00, -1.77367929e+00, -6.40216020e-04,
         3.91773788e-01,  1.43474944e+00,  4.90807345e-01,
        -5.14060973e-01, -5.57818354e-01],
       [ 5.54016785e-01,  6.54721152e-01,  1.01073841e+00,
         2.03357677e-01, -2.33272062e+00,  1.11412385e+00,
        -2.33455689e+00, -1.10847983e+00],
       [-3.66276460e-01, -4.09916504e-01,  1.60568780e-01,
         4.31232822e-01, -2.16052387e-01,  4.08589342e-02,
        -3.65023269e-01, -6.44544284e-01],
       [-4.33073917e-01, -5.92621182e-01,  2.83745441e-01,
         3.95551529e-01, -3.94060318e-02,  5.91218515e-01,
        -9.01550329e-01, -7.22958415e-01]])

In [17]:
v

array([[-1.05032823e+00, -1.77367929e+00, -6.40216020e-04,
         3.91773788e-01,  1.43474944e+00,  4.90807345e-01,
        -5.14060973e-01, -5.57818354e-01],
       [ 8.33586481e-01,  1.07788897e+00,  1.18697906e+00,
         1.70524693e-01, -2.98923181e+00,  1.22274163e+00,
        -2.65179257e+00, -1.20443691e+00],
       [-2.68188348e-01,  3.98829448e-01, -3.20563241e-01,
         6.60604046e-01, -5.35772117e-01, -1.36023480e+00,
         1.38422393e+00, -3.80478953e-01],
       [-6.95559727e-01,  9.22295153e-01, -3.04672214e-01,
         9.62962230e-01,  4.20468481e-01,  9.86192805e-01,
         3.07615529e-01, -5.26309583e-01]])

# All in One Function

In [18]:
def softmax(x):
  return (np.exp(x).T / np.sum(np.exp(x), axis=-1)).T

def scaled_dot_product_attention(q, k, v, mask=None):
  d_k = q.shape[-1]
  scaled = np.matmul(q, k.T) / math.sqrt(d_k)
  if mask is not None:
    scaled = scaled + mask
  attention = softmax(scaled)
  out = np.matmul(attention, v)
  return out, attention

In [19]:
values, attention = scaled_dot_product_attention(q, k, v, mask=mask)
print("Q\n", q)
print("K\n", k)
print("V\n", v)
print("New V\n", values)
print("Attention\n", attention)    

Q
 [[ 0.61273083  1.13588303 -0.07668819 -0.11967717 -1.10663414  0.72558852
   0.32466577  1.01415271]
 [-0.43102019  1.70190526 -0.27558612 -0.24166775  0.04095081  1.04663879
  -1.05256731  0.17359994]
 [ 0.06644399 -1.20945125 -0.24851811 -0.88769826 -1.0417973  -0.06049551
  -0.27082861  0.31942523]
 [-0.34797781  1.03270325 -0.21377292  1.36688     0.56332009 -1.53889172
  -0.18758204  1.19075472]]
K
 [[ 0.05174536 -1.17034938 -1.27582011  1.40638064 -0.87467139 -0.61068974
   1.50818084 -0.08137509]
 [ 1.5182199   0.29325469  0.37417365 -0.39781567  0.95559851  0.73178876
   0.05184995  0.43025465]
 [ 0.25307245 -1.30778832 -0.94060822 -0.5155473   1.293209    1.33223228
   1.47178584 -0.86005869]
 [-1.23924754 -1.46183783 -0.81514191 -0.59448902 -1.81793514  0.34715823
   0.15822686 -1.02893737]]
V
 [[-1.05032823e+00 -1.77367929e+00 -6.40216020e-04  3.91773788e-01
   1.43474944e+00  4.90807345e-01 -5.14060973e-01 -5.57818354e-01]
 [ 8.33586481e-01  1.07788897e+00  1.18697906e+0